# Optimal Cutting Parameters for Turning Operations with Costs of Quality and Tool Wear Compensation

This notebook calculates optimal process parameters for a turning operations using a model describes by Abdelmaguid and El-Hossainy, [[1]](http://iieom.org/ieom2012/pdfs/233.pdf) 
The problem described in that paper is one where multiple parts are turned with a single tool, and optimal cutting parameters are provided for multiple tool replacement scenarios, e.g., the tool is to replaced or reground after each part, after each second part, after 
each third part etc. The objective function seeks to optimize profit with constraints on quality (surface finish and dimensional accuracy).

In [7]:
import numpy as np
from scipy.optimize import minimize
import scipy.integrate as integrate
import functools # This allows us to curry functions.

### Cutting time

| Symbol     | Variable | Meaning | Relevant Concepts |Units|
|:----------:|:--------:|:-------:|-------------------|-----------------|
| $D$        | D       | Final product target diameter| [FinalDimension](http://modelmeth.nist.gov/manufacturing#FinalDimension), [Diameter](http://modelmeth.nist.gov/manufacturing#Diameter), [WorkpieceDimension](http://modelmeth.nist.gov/manufacturing#WorkpieceDimension) | [Millimeter](http://qudt.org/vocab/unit#Millimeter) |
| $d_i$      | d_i     |depth of cut for part *i* | [DepthOfCut](http://modelmeth.nist.gov/manufacturing#DepthOfCut)  | [Millimeter](http://qudt.org/vocab/unit#Millimeter) |
| $\delta_i$ | delta_i |tool wear compensation for part *i*| [DepthOfCut](http://modelmeth.nist.gov/manufacturing#DepthOfCut), [ToolWear](http://modelmeth.nist.gov/manufacturing#ToolWear) | [Millimeter](http://qudt.org/vocab/unit#Millimeter) |
| $V$        | V       |Cutting speed | [CuttingSpeed](http://modelmeth.nist.gov/manufacturing#CuttingSpeed) | [MeterPerMinute](http://qudt.org/vocab/unit#MeterPerMinute) |
| $f$        | f       | Feed rate | [CuttingFeedRate](http://modelmeth.nist.gov/manufacturing#CuttingFeedRate) | Millimeter/rev |
| $N$        | N       |Tool regrind scenario | [MachineToolChanger](http://modelmeth.nist.gov/manufacturing#MachineToolChanger), [TotalProcessValue](http://modelmeth.nist.gov/manufacturing#TotalProcessValue) | [Unitless](http://qudt.org/vocab/unit#Unitless) |

The depth of cut for part *i* is given by:

$ d_i = \frac{D_0 - D}{2} + \delta_i $

The mean cutting diameter is estimated as $\bar{D_i} = ( D_0 - d_i )$.

The cutting time for part *i* in minutes:

$ t_{c,i} = \frac{\pi \bar{D_i} L}{1000Vf} $

In [8]:
D   =  98.0     # target diameter, mm
D_0 = 100.0     # original diameter, mm
delta_i = 0.050 # tool wear compensation, mm, A DESIGN VARIABLE. I'm initializing it.
d_i = ((D_0 - D) / 2) + delta_i # depth of cut
D_bar_i = (D_0 - d_i) # mean cutting diameter
L = 35.0              # length of cut

**Function t_ci_calc** Cutting time for part i, a function of speed (m/min) and feed rate (mm/rev):

In [9]:
def t_ci_calc (V, f):   
    return ((np.pi * D_bar_i * L) / (1000.0 * V * f)) # Eqn 2

Just to assess the correctness of other calculations, we'll need a value for $t_c$ (time, cutting)for i=1 using the optimal values for V and f from Table 6 of the paper: V=70, f=0.0898. We can use this $t_{c,i}$ for calculations such as Eqn. 5 from the paper (tool wear)

In [10]:
t_0test = t_ci_calc(70.0, 0.0898)
t_0test

1.7308496273536194

$T_n = \sum\limits_{i=1}^n t_{ci}$

In [11]:
def T_n_calc(V,f,N): # POD I think this can just be N*t_ci_calc
    return(N*t_ci_calc(V,f))

### Tool Wear

| Symbol     | Variable | Meaning | Relevant Concepts | Units |
|-----------------------------------------------------|
| $W_i$      | W_i      | Total tool wear reached at the end of processing part i| [ToolWear](http://modelmeth.nist.gov/manufacturing#ToolWear), [TotalProcessValue](http://modelmeth.nist.gov/manufacturing#TotalProcessValue) | [Millimeter](http://qudt.org/vocab/unit#Millimeter) |
| $w_i$      | w_i      | Tool wear at time t for part i | [ToolWear](http://modelmeth.nist.gov/manufacturing#ToolWear), [IndividualPartValue](http://modelmeth.nist.gov/manufacturing#IndividualPartValue) | [Millimeter](http://qudt.org/vocab/unit#Millimeter) |
| $t_{w,i}$  | t_wi     | Time required to wear tool to point where part i-1 is finished| [Cumulative Use](http://modelmeth.nist.gov/manufacturing#CumulativeUse), [TotalProcessValue](http://modelmeth.nist.gov/manufacturing#TotalProcessValue), [ProcessTime](http://modelmeth.nist.gov/manufacturing#ProcessTime) | [Second](http://qudt.org/vocab/unit#Second) |
| $\hat{W}$  | W_hat    | Maximum allowed tool wear | [ToolWear](http://modelmeth.nist.gov/manufacturing#ToolWear), [MaximumAcceptedValue](http://modelmeth.nist.gov/manufacturing#MaximumAcceptableValue)| [Millimeter](http://qudt.org/vocab/unit#Millimeter) |
| $\Delta_i$ | Delta_i  | Workpiece diameter drift | [Drift](http://modelmeth.nist.gov/manufacturing#Drift) | [Millimeter](http://qudt.org/vocab/unit#Millimeter) |
| $\Theta$   | Theta    | Tool clearance angle | [ToolClearanceAngle](http://modelmeth.nist.gov/manufacturing#ToolClearanceAngle) | [DegreeAngle](http://qudt.org/vocab/unit#DegreeAngle) |
| $\Phi_i$ |  | Surface Roughness at time t for part i | [SurfaceRoughness](http://modelmeth.nist.gov/manufacturing#SurfaceRoughness), [CalculatedValue](http://modelmeth.nist.gov/manufacturing#CalculatedValue) | [Micrometer](http://qudt.org/vocab/unit#Micrometer) |
|$F_i$ |  | Cutting Force at time t for part i | [CuttingForce](http://modelmeth.nist.gov/manufacturing#CuttingForce), [CalculatedValue](http://modelmeth.nist.gov/manufacturing#CalculatedValue) | [Newton](http://qudt.org/vocab/unit#Newton) |

Empirically estimated tool wear formula: $w = k_wV^{\alpha_w}f^{\beta_w}d^{\gamma_w}t^{\sigma_w}$

It appears then, that tool wear is a function of t. If so, it might have better to use $w_i(t)$ in the paper. Nonetheless, sic, Equation 5:

$w_i = k_w V^{\alpha_w} f^{\beta_w} d_i^{\gamma_w} (t_{w,i} + t)^{\sigma_w}$  

for $0 \le t \le t_{c,i}$

The paper provides the following values for the parameters, and wear limit:

In [12]:
k_w = 8.2961e-5
alpha_w = 2.747 # paper has 1.261. 
beta_w = 1.473 
gamma_w = 1.261
sigma_w = 0.43
W_hat = 0.40
Theta = 15 #angle in degrees, not radians

**Function t_wi_calc** Eqn 6, cumulative time for wear equivalent to processing part i-1 **Is this really useful??? **

\begin{equation}
t_{w,i} = (\frac{1}{k_w} V^{-\alpha_w} f^{-\beta_w} d_i^{-\gamma_w} W_{i-1})^{\frac{1}{\sigma_w}}
\end{equation}

In [13]:
def t_wi_calc (i,V,f): # POD Is this really useful? Phi_calc just needs a time!
    if (i == 1):
        return(0.0)
    else:
        return((1/k_w * V**(-alpha_w) * 
                f**(-beta_w) * 
                d_i**(-gamma_w) * 
                w_i_calc(i-1,t_ci_calc(V,f),V,f))**(1/sigma_w))

**Function w_i_calc** : Tool wear processing part i after time t.

In [14]:
def w_i_calc (i,t,V,f):
    return(k_w*(V**alpha_w)*(f**beta_w)*(d_i**gamma_w)*((t_wi_calc(i,f,V)+t)**sigma_w))

In [15]:
# Let's test it with N=7 parameters. Should get about the time required to do 6 parts.
t7_wear_time = t_wi_calc(7,55.23,0.0804)
t7_cut_time = 6*t_ci_calc(55.23,0.0804)
print(t7_wear_time, t7_cut_time) # So close. Is there an identity here I missed?

14.7012368326008 14.701236832600781


Here is another test using optimal test parameter. Tool wear limit is 0.4mm. Since I'm getting a little less than that, 0.37558, I'm guessing that we are bumping up against the quality loss, instead of this.

In [16]:
w_i_calc(1,t_0test,70.0,0.0898) # Example, wear in mm after 1 part using optimal parameters

0.3755826829941073

In [17]:
# Wear *starting* the 6th part, with N=6 parameters (a bit too high unless delta_i < 0.030)
test1 = w_i_calc(6,0.0,56.32,0.0823)
test2 = w_i_calc(5,t_ci_calc(56.32,0.0823),56.32,0.0823)
print( test1, test2) # 0.41395264627206946 0.4139526462720694

0.41395264627206946 0.4139526462720694


In [18]:
# Wear *starting* the 7th part, with N=7 parameters (a bit too high unless delta_i < 0.030)
test_start = w_i_calc(7,0.0,55.23,0.0804) 
test_end   = w_i_calc(6,t_ci_calc(55.23,0.0804),55.23,0.0804)
print( test_start, test_end) # 0.41395264627206946 0.4139526462720694

0.41759265743261154 0.41759265743261165


$\hat{N}$ is the number of parts produced before regrinding is required. 

$\hat{N} = argmax_{i=1,2...}(\hat{W} - W_i)$ for $\hat{W} - W_i \ge 0$

Simpler formulation below:

In [19]:
def wear_limit_calc(N,V,f): # POD but this isn't used nonetheless!
    return w_i_calc(N,t_ci_calc(V,f),V,f)

In [20]:
test1 = wear_limit_calc(1,70.0,0.0898)
test2 = wear_limit_calc(1,70.0,0.0980)
print(test1, test2) # True False

0.3755826829941073 0.41142164644006035


Workpiece drift: $\Delta_i = 2(w_i - W_{i-1})tan(\Theta)$ for $0 \le t \le t_{c,i}$

In [21]:
def Delta_i_calc(i,t,V,f):
    if (i==1): 
        return(2.0 * w_i_calc(i,t,V,f) * np.tan(Theta))
    else:
        return(2.0 * (w_i_calc(i,t,V,f) - w_i_calc(i-1,t_ci_calc(V,f),V,f)) * np.tan(Theta))

In [22]:
Delta_i_calc(1,0.4,70.0,0.0898) # This is a little too high!

-0.34248397293218547

In [23]:
Delta_i_calc(2,0.4,70.0,0.1898)# Good

-0.26177712913418677

### Surface roughness

Surface roughness is determined empirically by the equation provided below:

$\Phi_i = k_r V^{\alpha_r} f^{\beta_r} d_i^{\gamma_r}(t_{w,i} + t)^{\sigma_r}$

The paper provides the following values for the parameters:

In [24]:
k_r = 11.619
alpha_r = 0.261
beta_r = 0.565
gamma_r = 0.565 # suspect!
sigma_r = 0.08887

In [25]:
def Phi_i_calc(i,t,V,f): 
    return(k_r * V**alpha_r * f**beta_r * d_i**gamma_r * (t_wi_calc(i,V,f) + t)**sigma_r)

In [26]:
# Paper says that this should be 9.203. 
# It seems likely that the parameters provided above are not correct.
test0 = Phi_i_calc(1,t_ci_calc(60.0, 0.0898),60.0,0.0898)
test1 = Phi_i_calc(1,t_ci_calc(70.0, 0.0898),70.0,0.0898)
test2 = Phi_i_calc(1,t_ci_calc(80.0, 0.0898),80.0,0.0898)

test3 = Phi_i_calc(1,t_ci_calc(70.0, 0.1898),70.0,0.1898)
test4 = Phi_i_calc(1,0.01,70.0,0.1898)

print (test0, test1, test2, test3, test4) # Good

9.48337623888521 9.73837584500662 9.964802273159505 13.907153338994608 9.401748980904973


### Cutting Force

The cutting force can be estimated empirically by the formula provided in the paper:

$ F_i = k_c V^{\alpha_c} f^{\beta_c} d_i^{\gamma_c}(t_{w,i} + t)^{\sigma_c}$

The paper provides the following values for the parameters:

In [27]:
k_c = 1514.6     # Paper says 173226.613 
alpha_c = 0.0992 # Paper says -0.992. Negative value doesn't make sense.
beta_c = 1.016
gamma_c = 1.033
sigma_c = 0.03877

In [28]:
def F_i_calc (i,t,V,f):
    return(k_c * V**alpha_c * f**beta_c * d_i**gamma_c * (t_wi_calc(i,V,f) + t)**sigma_c)

In [29]:
# Paper says 15kW max. I suppose (for no good reason) the numbers calculated are in watts.
# It seems to behave reasonably with respect to time (increasing slightly).
# It seems to behave reasonably with respect to feed (increasing substantially).
# It DOES NOT increase with increasing speed. In fact, speeds < 74 drive it too high!
# Because of these problems, I changed k_c and alpha_c above and checked a few values.
63.14 * F_i_calc(3,0.9,63.14,0.0856) 

13283.820873186105

In [30]:
x0 = 70.0 # Stuff for testing
x1 = 0.0898

In [31]:
x0*F_i_calc(1,T_n_calc(x0,x1,1),x0,x1) # Yes! over on power

14999.760909960267

### Economics

All of the following are candidates for information one might get from an ontology or "enterprise data."

| Parameter | Variable | Meaning | Relevant Concepts | Units |
|-----------------------|
| $C_h$     | C_h | Labor cost for loading/unloading per unit time | [Cost](http://modelmeth.nist.gov/manufacturing#Cost) | \$/min |
| $C_w$     | C_w | Labor cost for manning machine per unit time | [Cost](http://modelmeth.nist.gov/manufacturing#Cost) | \$/min |
| $C_z$     | C_z | Cost of machine per unit time in-cut time | [Cost](http://modelmeth.nist.gov/manufacturing#Cost) | \$/min |
| $C_g$     | C_g | Cost of regrinding a tool | [Cost](http://modelmeth.nist.gov/manufacturing#Cost) | [USDollar](http://qudt.org/vocab/unit#USDollar) |
| $p$       | p   | Revenue per final part | [Revenue](http://modelmeth.nist.gov/manufacturing#Revenue) | [USDollar](http://qudt.org/vocab/unit#USDollar) |
| $t_h$     | t_h | Loading and unloading time for a workpiece | [IndividualPartValue](http://modelmeth.nist.gov/manufacturing#IndividualPartValue), [ProcessTime](http://modelmeth.nist.gov/manufacturing#ProcessTime) | [Minute](http://qudt.org/vocab/unit#Minute) |
| $R$       | R   | Target surface roughness | [SurfaceRoughness](http://modelmeth.nist.gov/manufacturing#SurfaceRoughness), [TargetValue](http://modelmeth.nist.gov/manufacturing#TargetValue)| [Micrometer](http://qudt.org/vocab/unit#Micrometer) |
| $Q_c$     | Q_c | Quality loss | [Cost](http://modelmeth.nist.gov/manufacturing#Cost) | [USDollar](http://qudt.org/vocab/unit#USDollar) |
| $\ell_d$  | scriptl_d | Quality loss factor for diameter | [Cost](http://modelmeth.nist.gov/manufacturing#Cost), [Diameter](http://modelmeth.nist.gov/manufacturing#Diameter) | \$/mm^2 |
| $\ell_r$  | scriptl_r | Quality loss factor for roughness | [Cost](http://modelmeth.nist.gov/manufacturing#Cost), [SurfaceRoughness](http://modelmeth.nist.gov/manufacturing#SurfaceRoughness) | \$/mm^2 |

In [32]:
p = 37.5 # revenue per part
N = 1    # regrind tool after every part
C_h = 0.5  # Labor cost, unloading / min
C_w = 0.5  # Labor cost, manning the machine
C_z = 2.0  # Cost of machine while cutting
t_h = 2.0  # Part loading / unloading time / minute
C_g = 20.0 # Cost of regrinding 
GC = C_g   # Grinding cost, again.
scriptl_d = 125.0   # quality loss factor, diameter $/mm^2
scriptl_r = 0.0075  # quality loss factor, roughness $/um^2
# PROBLEM? Optimal values give \Phi = 9.7. Even they (Table 6) show 9.2. 
R = 2.5 # target roughness in microns -- so they will carry some loss here. 

Direct cost, labor cost for loading/unloading, machine and operator (a function of cutting time).

$ DC = N C_h t_h + (C_w + C_z) T_N$

In [33]:
def DC_calc (V, f, N): # Direct cost
    return (N * C_h * t_h + (C_w + C_z) * N * t_ci_calc(V, f))
def E_calc (N,p): # Total earnings per regrind cycle
    return (N*p)  

The quality constraint is a weighted sum of Taguchi loss functions for dimensional accuracy and surface roughness.

$QC = \sum\limits_{i=1}^n \ell_d\int_{T_{i-1}}^{T_i} \! (2\delta_i - \Delta_i)^2  \, \mathrm{d}t
+ \sum\limits_{i=1}^n \ell_r\int_{T_{i-1}}^{T_i} \! (\Phi_i - R)^2  \, \mathrm{d}t
$

In [34]:
def integrand1 (i,V,f,t):
    return (2.0 * delta_i - Delta_i_calc(i,t,V,f))**2
def integrand2 (i,V,f,t):
    return (Phi_i_calc(i,t,V,f) - R)**2
def QC_calc (V,f,N): 
    res = 0.0
    t = t_ci_calc(V,f)
    for i in range (1,N+1):
        res += scriptl_d*integrate.quad(functools.partial(integrand1,i,V,f),0.0,t)[0] + \
               scriptl_r*integrate.quad(functools.partial(integrand2,i,V,f),0.0,t)[0]  
    return res

In [35]:
t1 = QC_calc(70.0,0.0898,1)
t2 = QC_calc(80.0,0.0898,1)
print(t1, t2)
# Good, increase speed, quality penalty increases

70.2577127145984 103.72850870408936


In [36]:
V = 70.0
f = 0.0898
t = t_ci_calc(V,f)
t1= scriptl_d*integrate.quad(functools.partial(integrand1,1,V,f),0.0,t)[0] 
V = 80.0
t2= scriptl_d*integrate.quad(functools.partial(integrand1,1,V,f),0.0,t)[0] 
print( t1, t2)
# Quality penalty weighed more heavily than dimensional tolerance. 

69.71176972483444 130.39781510521442


In [37]:
V = 70.0
f = 0.0898
t = t_ci_calc(V,f)
t1 = scriptl_r*integrate.quad(functools.partial(integrand2,1,V,f),0.0,t)[0] 
V = 90.0
#f = 0.1898
t = t_ci_calc(V,f)
t2 = scriptl_r*integrate.quad(functools.partial(integrand2,1,V,f),0.0,t)[0] 
print( t1, t2)
# BAD? V increasing, roughness penalty decreasing.
# Is it really bad? Maybe not -- at V=90 the loss is over a shorter period.
# It is more sensitive (and monotonic?) WRT increasing feed rate.

0.5459429897639597 0.47813518185065085


### Profit

Profit is earnings minus direct cost, regrind cost, and quality penalty.

$P = E - (DC + GC + QC)$ 

Earnings is number processed times revenue per part.

$E = N p$

In [38]:
def P_calc(V,f,N):
    return N*p - (DC_calc(V,f,N) + GC + QC_calc(V,f,N))

In [39]:
P_calc(70.0,0.0898,1)

-58.08483678298245

### Constraints

$f_{min} \le f \le f_{max}$ mm/rev

$V_{min} \le V \le V_{max}$ m/min

$d_{min} \le d_i \le d_{max}$  $ \forall i=1,2,3...$  mm

The paper provides the following bounds on these variables:

In [40]:
f_min = 0.08
f_max = 0.28
V_min = 32.0
V_max = 70.0
d_min = 0.5
d_max = 1.5
power_max = 15000.0 # watts -- 15 Kw in the paper 

| Symbol | Variable | Meaning | Relevant Concepts | Units |
|------------------|
| $D_{LSL}$ | D_USL | Lower bound on final part diameter| [FinalDimension](http://modelmeth.nist.gov/manufacturing#FinalDimension), [Diameter](http://modelmeth.nist.gov/manufacturing#Diameter), [WorkpieceDimension](http://modelmeth.nist.gov/manufacturing#WorkpieceDimension), [MinimumAcceptable Value](http://modelmeth.nist.gov/manufacturing#MinimumAcceptableValue)| [Millimeter](http://qudt.org/vocab/unit#Millimeter) |
| $D_{USL}$ | D_LSL |Upper bound on final part diameter|[FinalDimension](http://modelmeth.nist.gov/manufacturing#FinalDimension), [Diameter](http://modelmeth.nist.gov/manufacturing#Diameter), [WorkpieceDimension](http://modelmeth.nist.gov/manufacturing#WorkpieceDimension), [MaximumAcceptableValue](http://modelmeth.nist.gov/manufacturing#MaximumAcceptableValue) | [Millimeter](http://qudt.org/vocab/unit#Millimeter) |
| $R_{USL}$ | R_USL | Maximum allowable surface roughness | [SurfaceRoughness](http://modelmeth.nist.gov/manufacturing#SurfaceRoughness), [MaximumAcceptableValue](http://modelmeth.nist.gov/manufacturing#MaximumAcceptableValue) | [Micrometer](http://qudt.org/vocab/unit#Micrometer) |

The paper provides the following values:

In [41]:
R_USL = 10.0
D_LSL = 97.9
D_USL = 98.1

The objective function (profit per minute):

$Z = \frac{P}{N t_h + T_N}$ 

The design variables need to be in a vector. `x[0] = V, x[1] = f`.

In [42]:
V = 70.0
f = 0.0898
P_calc(V,f,N)/(N*t_h + T_n_calc(V,f,N)) # Profit / unit time -- paper shows $3.01

-15.568796007515159

In [43]:
N = 1
cons = ({'type': 'ineq',
         'fun' : lambda x: np.array([power_max -
                                     x[0]*F_i_calc(N,T_n_calc(x[0],x[1],N),x[0],x[1])])},
        {'type': 'ineq',
         'fun' : lambda x: np.array([R_USL - 
                                     Phi_i_calc(N,T_n_calc(x[0],x[1],N),x[0],x[1])])},
        # POD I *think* drift will be highest on end of Nth part.
        {'type': 'ineq',
         'fun' : lambda x: np.array([D_USL -
                                     Phi_i_calc(N,T_n_calc(x[0],x[1],N),x[0],x[1])])},        
        {'type': 'ineq',
         'fun' : lambda x: np.array([Phi_i_calc(N,T_n_calc(x[0],x[1],N),x[0],x[1])
                                    - D_LSL])},
        {'type': 'ineq',
         'fun' : lambda x: np.array([W_hat - 
                                     wear_limit_calc(N,x[0],x[1])])})
                                                                         
def obj_func (x,sign=-1): 
    return sign*(P_calc(x[0],x[1],N)/(N*t_h + T_n_calc(x[0],x[1],N)))    

res = minimize(obj_func, [60.0,0.08], bounds=[(V_min, V_max), (f_min, f_max)],
               constraints=cons,
               method='SLSQP', options={'disp': True})
print(res.x) 

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 1.19554040593
            Iterations: 15
            Function evaluations: 57
            Gradient evaluations: 11
[ 32.           0.12599559]


In [50]:
N = 1
cons = ({'type': 'ineq',
         'fun' : lambda x: np.array([wear_limit_calc(N,x[0],x[1])
                                     - W_hat])})                           
def obj_func (x,sign=-1): 
    return sign*(P_calc(x[0],x[1],N)/(N*t_h + T_n_calc(x[0],x[1],N)))    

res = minimize(obj_func, [60.0,0.08], bounds=[(V_min, V_max), 
                                              (f_min, f_max)],
               constraints=cons,
               method='SLSQP', options={'disp': True})

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 9.30666289486
            Iterations: 9
            Function evaluations: 39
            Gradient evaluations: 9


In [45]:
N = 1
cons = ({'type': 'ineq',
         'fun' : lambda x: np.array([wear_limit_calc(N,x[0],x[1])
                                     - W_hat])})                                                                         
def obj_func (x,sign=-1): 
    return sign*(P_calc(x[0],x[1],N)/(N*t_h + T_n_calc(x[0],x[1],N)))    

res = minimize(obj_func, [60.0,0.08], bounds=[(V_min, V_max), 
                                              (f_min, f_max)],
               constraints=cons,
               method='SLSQP', options={'disp': True})

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 9.30666289486
            Iterations: 9
            Function evaluations: 39
            Gradient evaluations: 9


In [46]:
obj_func(res.x)

9.306662894858027

In [47]:
res.x[0]*F_i_calc(N,T_n_calc(res.x[0],res.x[1],N),res.x[0],res.x[1])

27257.341922520467

In [48]:
Phi_i_calc(1,T_n_calc(res.x[0],res.x[1],N),res.x[0],res.x[1])

15.395805209834789

In [49]:
obj_func(np.array([x0,x1])) # It is going to 65 because 70 isn't the optimum anymore.

15.568796007515159

## References

1. Abdelmaguid, T. and F., El-Hossainy, T. J., *Optimal Cutting Parameters for Turning Operations with Costs of Quality and Tool Wear Compensation*, Proceedings of the 2012 International Conference on Industrial Engineering and Operations Management, Istambul, Turkey, July 3--6, 2012.